# gebrauchte Bücher aus Goodreads to-read liste suchen

In [1]:
import numpy as np
import csv
import os
import datetime as dt
from matplotlib import pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
TOTAL_NUMBER = 300  # max number of books to search on Booklooker
START_DATE = pd.Timestamp('2020-01-01') # Books added to to-read shelf after this date will be considered
DATE = str(dt.datetime.today().date()) # in- and output datestamp
print(DATE)

2022-01-01


# WIP
## goodreads export csv

In [3]:
# #Simple assignment
# from selenium.webdriver import Edge

# driver = Edge()
# #Or use the context manager
# from selenium.webdriver import Edge

# with Edge() as driver:
#    #your code inside this indent
#    pass


## goodreads CSV-Datei parsen

In [4]:
csv_path = os.path.join(os.getcwd(), f'goodreads_library_export_{DATE}.csv')

df = pd.read_csv(csv_path, index_col='Book Id',
dtype={"Title": str, "Author": str, "Author l-f": str, "Additional Authors": str, "ISBN":str,"ISBN-13":str, "Publisher": str, "Binding": str, 'Bookshelves': 'category', 'Exclusive Shelf': 'category', },
parse_dates=["Original Publication Year", 'Date Read', 'Date Added', "Year Published"])
df.head(10)

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
1508423,Was vom Tage übrigblieb,Kazuo Ishiguro,"Ishiguro, Kazuo",NaN,"=""344273309X""","=""9783442733095""",0,4.14,btb,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
33312081,Die drei Sonnen (Die drei Sonnen #1),Liu Cixin,"Cixin, Liu",Martina Hasse,"=""""","=""""",0,4.06,Heyne Verlag,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
44245751,"How To: Wie man's hinkriegt: Absurde, wirklich...",Randall Munroe,"Munroe, Randall",Ralf Pannowitsch,"=""3328600914""","=""9783328600916""",0,4.11,Penguin Deutschland,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
54184721,Human Compatible: Künstliche Intelligenz und w...,Stuart Russell,"Russell, Stuart",NaN,"=""""","=""""",0,4.15,NaN,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29352085,Tipping Point: Wie kleine Dinge Großes bewirke...,Malcolm Gladwell,"Gladwell, Malcolm",Malte Friedrich,"=""""","=""""",0,3.98,Goldmann Verlag,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
42983724,Drive Your Plow Over the Bones of the Dead,Olga Tokarczuk,"Tokarczuk, Olga",Antonia Lloyd-Jones,"=""0525541330""","=""9780525541332""",0,3.96,Riverhead Books,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
42299635,Das Zeitalter des Überwachungskapitalismus,Shoshana Zuboff,"Zuboff, Shoshana",NaN,"=""""","=""9783593509303""",0,4.07,Campus Verlag GmbH,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
13626909,Das Omnivoren-Dilemma: Wie sich die Industrie ...,Michael Pollan,"Pollan, Michael",Petter Cobbe,"=""3442219337""","=""9783442219339""",0,4.16,Goldmann TB,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3030362,Die Schock-Strategie,Naomi Klein,"Klein, Naomi",NaN,"=""3100396111""","=""9783100396112""",0,4.23,Fischer,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(429, 30)

In [6]:
df['ISBN'] = df['ISBN'].str.split('\"').str.get(1)
df['ISBN13'] = df['ISBN13'].str.split('\"').str.get(1)
df.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
1508423,Was vom Tage übrigblieb,Kazuo Ishiguro,"Ishiguro, Kazuo",NaN,344273309X,9783442733095,0,4.14,btb,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
33312081,Die drei Sonnen (Die drei Sonnen #1),Liu Cixin,"Cixin, Liu",Martina Hasse,,,0,4.06,Heyne Verlag,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
44245751,"How To: Wie man's hinkriegt: Absurde, wirklich...",Randall Munroe,"Munroe, Randall",Ralf Pannowitsch,3328600914,9783328600916,0,4.11,Penguin Deutschland,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
54184721,Human Compatible: Künstliche Intelligenz und w...,Stuart Russell,"Russell, Stuart",NaN,,,0,4.15,NaN,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
29352085,Tipping Point: Wie kleine Dinge Großes bewirke...,Malcolm Gladwell,"Gladwell, Malcolm",Malte Friedrich,,,0,3.98,Goldmann Verlag,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [7]:
to_read = df[df['Exclusive Shelf'] == 'to-read']


In [8]:
to_read.sort_values('Date Added', ascending=False, inplace=True)
to_read_2021 = to_read[to_read['Date Added'] > START_DATE]

C:\Users\simon\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
to_read_2021.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
1508423,Was vom Tage übrigblieb,Kazuo Ishiguro,"Ishiguro, Kazuo",NaN,344273309X,9783442733095,0,4.14,btb,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
942450,Der Mythos des Sisyphos,Albert Camus,"Camus, Albert",NaN,3499227657,9783499227653,0,4.17,Rowohlt Tb.,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
58931278,Allein,Daniel Schreiber,"Schreiber, Daniel",NaN,,9783446267923,0,4.16,Hanser Berlin,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
58515679,Die Anomalie,Hervé Le Tellier,"Tellier, Hervé Le",NaN,3498002589,9783498002589,0,3.89,Rowohlt Verlag,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1414809,Der Meister und Margarita,Mikhail Bulgakov,"Bulgakov, Mikhail","Michail Bulgakow, Thomas Reschke",,,0,4.29,Luchterhand Literaturverlag,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [10]:
type(to_read_2021)

pandas.core.frame.DataFrame

## Beispiel
### Jetzt Suchmaschine mittels BeatifulSoup scrapen um Bestpreise für Bücher zu finden

In [11]:
#Testbuch
book = to_read_2021.iloc[6]
book

Title                         Die Macht der Geographie im 21. Jahrhundert: 1...
Author                                                            Tim  Marshall
Author l-f                                                        Marshall, Tim
Additional Authors                                                          NaN
ISBN                                                                 3423283017
ISBN13                                                            9783423283014
My Rating                                                                     0
Average Rating                                                             4.21
Publisher                                               dtv Verlagsgesellschaft
Binding                                                               Paperback
Number of Pages                                                           416.0
Year Published                                              2021-01-01 00:00:00
Original Publication Year               

In [12]:
url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/isbn=9783446267718?sortOrder=preis_total'
search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
base_url = 'https://www.booklooker.de'
query_parameters = {
    'isbn': book['ISBN13'],
    'sortOrder': 'preis_total'
}

## Kann geskippt werden

In [13]:
page = requests.get(url)
soup = BeautifulSoup(page.content)

table class="resultlist_products"

tbody

tr id="article_*ID*" class="articleRow resultlist_products*product*" ODER

articleRow resultlist_products*premium*

In [14]:
results = soup.find(class_='resultlist_products')
products = results.find_all('tr', class_='articleRow')


In [15]:
entries = []
for idx, product in enumerate(products):
    product_info = product.find("td", class_="resultlist_productsimage")
    price_info = product.find("td", class_="resultlist_productsright")
    title_info =product_info.find('h2', class_='unformated')
    book_title = title_info.find('span', class_='artikeltitel')
    link = base_url + title_info.a.attrs['href']

    author = product_info.find('span', class_='notranslate')
    condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
    if condition:
        prefix = 'Zustand: '
        condition = condition[len(prefix):] if condition.startswith(prefix) else condition
    else:
        condition = str(product_info.find(string=lambda text: 'neuware' in text.lower()))
    price = price_info.find('span', class_='price')
    row = [book_title.text.strip(), author.text.strip(), price.text.strip()[:-2], condition.strip(), link]
    entries.append(row)
    # print(f"[{idx}]: title: {book_title.text.strip()}, author: {author.text.strip()}, price = {price.text.strip()}, state: {state.strip()}), link: {link}")


In [16]:
web = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'Condition', 'URL'])
web.head()

,Title,Author,Price,Condition,URL
0,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","13,00",deutliche Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...
1,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","13,40",leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...
2,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","14,00",wie neu,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...
3,"Dreck - Wie ich meine Familie einpackte, Koch ...","Dreck - Wie ich meine Familie einpackte, Koch ...","16,90",gebraucht; sehr gut,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...
4,"Dreck: Wie ich meine Familie einpackte, Koch i...","Dreck: Wie ich meine Familie einpackte, Koch i...","18,58",gebraucht; sehr gut,https://www.booklooker.de/B%C3%BCcher/Bill-Buf...


## Funktionen
Build Search URL

In [17]:
from urllib.parse import urlencode, urljoin, quote_plus
def build_booklooker_url(**kwargs):
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['autor'] = author
        payload['titel'] = title
        # print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_abebooks_url(**kwargs):
    #TODO
    raise NotImplementedError('not yet finished.')
    search_url = 'https://www.abebooks.de/servlet/SearchResults?sts=t&cm_sp=SearchF-_-home-_-Results&an=Author&tn=Title&kn=&isbn='
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['an'] = author
        payload['tn'] = title
        # print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_url(base_url, payload):
    result = urlencode(payload, quote_via=quote_plus)
    url = urljoin(base_url, result)
    return url

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('utf-8', 'ignore'))
    return soup

In [18]:
import re
non_decimal = re.compile(r'[^\d.]+')

Soup-Suchergebnisse auslesen

In [19]:
def get_booklooker_results(soup, limit=10, as_df=False, rating=None):
    entries = []
    results = soup.find(class_='resultlist_products')
    if results is None:
        return entries
    products = results.find_all('tr', class_='articleRow')
    for idx, product in enumerate(products):
        try:
            product_info = product.find("td", class_="resultlist_productsimage")
            title_info =product_info.find('h2', class_='unformated')
            book_title = title_info.find('span', class_='artikeltitel notranslate')
            link = base_url + title_info.a.attrs['href']

            author_info =product_info.find('h3', class_='unformated')
            author = author_info.find('span', class_='notranslate')

            condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
            prefix = 'Zustand: '
            condition = condition[len(prefix):] if condition.startswith(prefix) else condition
            
            price_info = product.find("div", class_="productPrices")
            prices = price_info.find_all(string=lambda text: '€' in text.lower())
        except AttributeError as notfound:
            continue
        price_list = [s[:-1].strip() for s in prices]
        price = price_list[0] if len(price_list) > 0 else ""
        vsk = price_list[1] if len(price_list) > 1 else ""
        price = price.replace(",", ".")
        vsk = vsk.replace(",", ".")
        price = non_decimal.sub('', price)
        vsk = non_decimal.sub('', vsk)
        price = float(price) if len(price) > 0 else 0
        vsk = float(vsk) if len(vsk) > 0 else 0
        row = (book_title.text.strip(), author.text.strip(), price, vsk, condition.strip(), link)
        entries.append(row)
        if idx > (limit - 1):
            break
    if as_df:
        df = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL'])
        if rating:
            df['Average Rating'] = rating
        return df
    else:
        return entries


## Bringing it all together
Für jedes Buch:
- URL erzeugen
- HTML-Suppe abrufen
- Suchergebnisse parsen
- DataFrame erzeugen

In [20]:
def get_sales_info_df(df, limit=10, as_df=True):
    url = build_booklooker_url(**df[['Title','Author']])
    url = build_booklooker_url(**df)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df, rating=df['Average Rating'])
    #TODO erst mit Titel + Autor suchen, dann ISBN. So fehlen viele Ergebnisse da eine spezifische Ausgabe!
    if len(search_results) < 1:
        print(f"no search results found for {df['Title']},{df['Author']}, trying ISBN")
        url = build_booklooker_url(**df.drop(columns=['Title', 'Author']) )
        soup = get_soup(url)
        search_results = get_booklooker_results(soup, limit=limit, as_df=as_df, rating=df['Average Rating'])

    if as_df:
        return search_results
    else:
        return np.array(search_results)

In [21]:
def get_sales_info(title, isbn, isbn13, limit=10, as_df=True):
    isbn_ = isbn13 if len(isbn13) > 0 else isbn
    if len(isbn_) <= 0:
        print(f'{title}: no ISBN found!')
        return []
        # raise KeyError('No ISBN Found')
    d = {'ISBN': isbn_}
    url = build_booklooker_url(**d)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    return np.array(search_results)

In [22]:
# sales_info = []
my_df = to_read_2021.iloc[:TOTAL_NUMBER]
# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
sales_infos = my_df.progress_apply(get_sales_info_df, axis=1)


  0%|          | 0/116 [00:00<?, ?it/s]

no search results found for The 12 Week Year,Brian P. Moran, trying ISBN
no search results found for The Comfort Book,Matt Haig, trying ISBN
no search results found for On Democracy,Robert A. Dahl, trying ISBN
no search results found for Ahora me rindo y eso es todo,Álvaro Enrigue, trying ISBN
no search results found for A Thousand Brains: A New Theory of Intelligence,Jeff Hawkins, trying ISBN
no search results found for Speed & Scale: A Global Action Plan for Solving Our Climate Crisis Now,John Doerr, trying ISBN
no search results found for Rationality: What It Is, Why It Seems Scarce, Why It Matters,Steven Pinker, trying ISBN
no search results found for From Dawn to Decadence: 500 Years of Western Cultural Life, 1500 to the Present,Jacques Barzun, trying ISBN
no search results found for Numbers Don't Lie: 71 Things You Need to Know About the World,Vaclav Smil, trying ISBN
no search results found for The Atom Station,Halldór Laxness, trying ISBN
no search results found for Missing Out

In [23]:
sales_infos.iloc[7]

[]

In [24]:
sales_infos = sales_infos.apply(lambda l: pd.DataFrame(l) if type(l)==list else l)

In [25]:
sales_infos.head()
# nd = np.array(sales_infos)

Book Id
1508423                                                   ...
942450                                                    ...
58931278        Title            Author  Price  VSK Condit...
58515679                     Title             Author  Pri...
1414809                                                   ...
dtype: object

In [26]:
sales_infos.iloc[5].iloc[0]

Title             Die Macht der Geographie - Wie sich Weltpoliti...
Author                                                 Tim Marshall
Price                                                           8.9
VSK                                                            2.25
Condition                                   leichte Gebrauchsspuren
URL               https://www.booklooker.de/B%C3%BCcher/Tim-Mars...
Average Rating                                                 4.19
Name: 0, dtype: object

Erstmal speichern

In [27]:
pkl_fn = f'sales_infos_2021_recent{TOTAL_NUMBER}_{DATE}.pkl'


In [28]:
sales_infos.to_pickle(pkl_fn)

In [29]:
sales_infos = pd.read_pickle(pkl_fn)


In [30]:
sales_infos.iloc[6]

,Title,Author,Price,VSK,Condition,URL,Average Rating
0,Die Macht der Geographie im 21. Jahrhundert - ...,Tim Marshall,16.9,2.4,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Tim-Mars...,4.21
1,Die Macht der Geographie im 21. Jahrhundert - ...,Tim Marshall,19.5,3.0,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Tim-Mars...,4.21
2,Die Macht der Geographie im 21. Jahrhundert - ...,Tim Marshall,24.0,0.0,None,https://www.booklooker.de/B%C3%BCcher/Neuware/...,4.21


In [31]:
dic = sales_infos.to_dict()
sales_df = pd.concat(dic)

In [32]:
sales_df.iloc[6].iloc[0]

'Was vom Tage übrigblieb. Roman'

In [33]:
book_ids = my_df.index

In [34]:
sales_infos = sales_infos.apply(lambda l: pd.DataFrame(l) if type(l)==list else l)


In [35]:
sales_infos.iloc[0]

,Title,Author,Price,VSK,Condition,URL,Average Rating
0,Was vom Tage übrigblieb,Kazuo Ishiguro,2.00,2.1,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
1,Was vom Tage übrigblieb,Kazuo Ishiguro,2.00,2.1,wie neu,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
2,"Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...","Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...",2.99,2.2,wie neu,https://www.booklooker.de/B%C3%BCcher/Kazuo-an...,4.14
3,Was vom Tage übrigblieb. Roman,Kazuo Ishiguro,3.37,2.1,deutliche Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
4,Was vom Tage übrigblieb,Kazuo Ishiguro,3.45,2.4,leichte Gebrauchsspuren,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
5,Was vom Tage übrigblieb,Kazuo Ishiguro,3.79,2.1,wie neu,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
6,Was vom Tage übrigblieb. Roman,"Ishiguro, Kazuo und Hermann Stiehl",6.48,0.0,gebraucht; gut,https://www.booklooker.de/B%C3%BCcher/Kazuo-un...,4.14
7,Was vom Tage übrigblieb : Roman.,Kazuo Ishiguro,4.50,2.5,gebraucht; sehr gut,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
8,Was vom Tage übrigblieb. Roman. Aus dem Englis...,Kazuo Ishiguro,5.40,2.5,None,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14
9,Was vom Tage übrigblieb,Kazuo Ishiguro,6.00,2.0,wie neu,https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...,4.14


In [36]:
sales_df.head()

Title  \
1508423 0                            Was vom Tage übrigblieb   
        1                            Was vom Tage übrigblieb   
        2  Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...   
        3                     Was vom Tage übrigblieb. Roman   
        4                            Was vom Tage übrigblieb   

                                                      Author  Price  VSK  \
1508423 0                                     Kazuo Ishiguro   2.00  2.1   
        1                                     Kazuo Ishiguro   2.00  2.1   
        2  Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...   2.99  2.2   
        3                                     Kazuo Ishiguro   3.37  2.1   
        4                                     Kazuo Ishiguro   3.45  2.4   

                           Condition  \
1508423 0    leichte Gebrauchsspuren   
        1                    wie neu   
        2                    wie neu   
        3  deutliche Gebrauchsspuren   
        4    leichte Gebrauchsspuren   

                                                         URL  Average Rating  
1508423 0  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14  
        1  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14  
        2  https://www.booklooker.de/B%C3%BCcher/Kazuo-an...            4.14  
        3  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14  
        4  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14

In [37]:
sales_df.dtypes

Title              object
Author             object
Price             float64
VSK               float64
Condition          object
URL                object
Average Rating    float64
dtype: object

In [38]:
# sales_df['Price'] = sales_df['Price'].apply(lambda x: x.replace(',','.'))
# sales_df['Price'] = sales_df['Price'].apply(lambda x: non_decimal.sub('', x))
# sales_df['VSK'] = sales_df['VSK'].apply(lambda x: x.replace(',','.'))
# sales_df['VSK'] = sales_df['VSK'].apply(lambda x: non_decimal.sub('', x))
# sales_df.loc[10614].loc[6]['URL']
# sales_df['Price'] = pd.to_numeric(sales_df['Price'])
# sales_df['VSK'] = pd.to_numeric(sales_df['VSK'])
sales_df['Total Price'] = sales_df.fillna(0)['Price'] + sales_df.fillna(0)['VSK']

In [39]:
sales_df['Condition'].replace('None', '', inplace=True)

In [40]:
sales_df.head()

Title  \
1508423 0                            Was vom Tage übrigblieb   
        1                            Was vom Tage übrigblieb   
        2  Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...   
        3                     Was vom Tage übrigblieb. Roman   
        4                            Was vom Tage übrigblieb   

                                                      Author  Price  VSK  \
1508423 0                                     Kazuo Ishiguro   2.00  2.1   
        1                                     Kazuo Ishiguro   2.00  2.1   
        2  Was vom Tage übrigblieb. Roman Ishiguro, Kazuo...   2.99  2.2   
        3                                     Kazuo Ishiguro   3.37  2.1   
        4                                     Kazuo Ishiguro   3.45  2.4   

                           Condition  \
1508423 0    leichte Gebrauchsspuren   
        1                    wie neu   
        2                    wie neu   
        3  deutliche Gebrauchsspuren   
        4    leichte Gebrauchsspuren   

                                                         URL  Average Rating  \
1508423 0  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14   
        1  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14   
        2  https://www.booklooker.de/B%C3%BCcher/Kazuo-an...            4.14   
        3  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14   
        4  https://www.booklooker.de/B%C3%BCcher/Kazuo-Is...            4.14   

           Total Price  
1508423 0         4.10  
        1         4.10  
        2         5.19  
        3         5.47  
        4         5.85

In [41]:
df_pkl_fn = f'sales_df_2021_recent{TOTAL_NUMBER}_{DATE}.pkl'

In [42]:
sales_df.to_pickle(df_pkl_fn)

In [43]:
#TODO ggf nochmal erweitern / joinen mit der goodreads Tabelle für Indexierung mittels Titel, etc. (vorallem bei fehelnden Ergebnissen)

In [44]:
sales_df = pd.read_pickle(df_pkl_fn)

In [45]:
sorted_df = sales_df.sort_values(by='Total Price')

## TODO: by Price: only keep lowest? or collapsable elements

In [46]:
by_avg_rating = sales_df.set_index(['Average Rating', sales_df.index, 'Total Price']).sort_index(ascending=[False,True,True])
by_total_price = sales_df.set_index(['Total Price','Average Rating']).sort_index(ascending=[True, False])
by_id = sales_df.set_index([sales_df.index, 'Total Price']).sort_index(ascending=[False, True])


TODO:

1. Sortieren innerhalb BookID
2. nur Bestpreis pro Buch (oder Top 3? oder 1 pro Zustand?)
3. In Flask o.ä. integrieren mit Template für geile Bootstrap-Tabelle mit Filter usw

In [47]:
def export_html(df, fn):
    pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

    html_string = '''
    <html>
    <head>
        <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">

    <!-- Bootstrap CSS -->
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-BmbxuPwQa2lc/FVzBcNJ7UAyJxM6wuqIj61tLrc4wSX0szH/Ev+nYRRuWlolflfl" crossorigin="anonymous">

    <title>HTML Pandas Dataframe with CSS</title></head>
    <link rel="stylesheet" type="text/css" href="df_style.css"/>
    <body>
        {table}
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta2/dist/js/bootstrap.bundle.min.js" integrity="sha384-b5kHyXgcpbZJO/tY9Ul7kGkf1S0CWuKcCD38l8YkeH8z8QjE0GmW1gYU5S9FOnJ0" crossorigin="anonymous"></script>

    </body>
    </html>.
    '''

    # OUTPUT AN HTML FILE
    with open(fn, 'w', encoding='utf-8') as f:
        f.write(html_string.format(table=df.to_html(render_links=True, classes=['table', 'table-hover'])))

In [48]:
export_html(by_avg_rating, f"report_by_rating_{DATE}.html")
export_html(by_total_price, f"report_by_price_{DATE}.html")
export_html(by_id, f"report_by_id_{DATE}.html")

Export to Excel

In [49]:
by_id.to_excel(f'report_{DATE}.xlsx')

In [50]:
export_html(by_avg_rating[by_avg_rating['Price'] < 10], f"cheap_{DATE}.html")

## Visualisierung
- HTML Dashboard (Integration in Flask, FastAPI, Django ?)
- oder super simpel mit Plotly und irgend ner Dashboard JS API (gibts sicher)
- Entweder einfache Tabelle, sortierbar usw.
- oder mit Plots: Barplots: Ein Balken für Neupreis, einen für Gebraucht (mit Whiskers)


In [51]:
def highlight_min(df, color='yellow'):
    '''
    highlight the minimum combined price.
    '''
    attr = 'background-color: {}'.format(color)
    is_min = df == df['Total Price'].min().min()
    return pd.DataFrame(np.where(is_min, attr, ''),
                            index=df.index, columns=df.columns)


In [52]:
# sales_df.style.apply(highlight_min, axis=None)

In [53]:
# sales_df